# Отток клиентов

Заказчик исследования - «Бета-Банк», откуда стали уходить клиенты. Клиенты уходят ежемесячно в небольших, но заметных количествах. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых. Входные данные от банка - данные о клиентах и их экономическом поведении, а также факты расторжения договоров с банком. Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. О качестве данных ничего неизвестно, поэтому потребуется подготовка данных.

Результаты исследования будут использованы заказчиком в коммерческих целях.

**Цель исследования**: построить модель, прогнозирующую вероятный уход клиента от банковского обслуживания. 

Ход исследования: 

Входные данные от банка - статистика клиентов за некоторый промежуток времени. Перед анализом данные необходимо подготовить и проверить на возможные проблемы. Далее акцентируем внимание на построение качественной модели для предсказания возможности оттока клиентов. Качествой модели определится предельно большим значением *F1*-меры. Дополнительно измерим *AUC-ROC* и сравним со значением *F1*-меры. По необходимости избавимся от дисбаланса классов в данных. После построения модели необходимо будет провести тестирование на тестовой выборке.

Таким образом, исследование пройдёт в 5 этапов:
* 1. Изучение данных из файла.
* 2. Подготовка данных.
* 3. Исследование задачи.
* 4. Борьба с дисбалансом.
* 5. Тестирование модели.



Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Изучение данных из файла

Перед тем, как приступить к изучению данных из файла, импортируем все нужные для текущего исследования библиотеки и модули:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

Также сохраним в начале исследования необходимые константные переменные:

In [2]:
RANDOM_STATE = 2808

Прочитаем полученные данные и сохраним их в таблице `df`, после чего выведем на экран первые 5 строк:

In [3]:
df = pd.read_csv('/datasets/Churn.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Получим общую информацию о таблице:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В таблице содержится  10000 наблюдений и 14 признаков. 8 признаков имеют тип данных `int`, 3 - `float`, 3 - `object`. Предварительно можно утверждать, что данных для построения модели достаточно. Согласно документации к данным:

* `RowNumber` — индекс строки в данных;
* `CustomerId` — уникальный идентификатор клиента;
* `Surname` — фамилия;
* `CreditScore` — кредитный рейтинг;
* `Geography` — страна проживания;
* `Gender` — пол;
* `Age` — возраст;
* `Tenure` — сколько лет человек является клиентом банка;
* `Balance` — баланс на счёте;
* `NumOfProducts` — количество продуктов банка, используемых клиентом;
* `HasCrCard` — наличие кредитной карты;
* `IsActiveMember` — активность клиента;
* `EstimatedSalary` — предполагаемая зарплата;
* `Exited` — факт ухода клиента

Целевым признаком является `Exited` - факт ухода клиента. Замена типов данных не потребуется, однако у нас есть 3 признака с типом `object`, что потребует перекодирования. Также в названиях признаков использован не лучший регистр для языка Python, а также присутствует верхний регистр. Избавимся от этого позднее.

Посмотрим на корреляции в данных. Для этого применим специальный метод библиотеки `pandas`:

In [5]:
df.corr()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.007322,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.021418,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005840,0.005308,1.000000,-0.003965,-0.000062,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.000783,0.009497,-0.003965,1.000000,-0.013134,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.007322,-0.021418,-0.000062,-0.013134,1.000000,-0.007911,0.011979,0.027232,-0.032178,0.010520,-0.016761
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.007911,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.011979,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.027232,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.032178,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.010520,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


Факт ухода связан с возрастом, о чём говорит корреляция 0.285 - наиболее сильная зависимость на уход клиента. Также выявили, что чем больше баланс на счёте клиента, тем меньше продуктов банка он использует (корреляция -0.304).

### Вывод

В таблице имеются данные по 10000 клиентам "Бета-Банка". По каждому из них имеется набор признаков, как об общей информации, так и об экономической активности и факте ухода от услуг банка. Можно утверждать, что выборка достаточна для построения качественной модели. В данных выявлено 2 значимые корреляции: уход клиента от услуг банка и его возраст (0.285) и баланс на счёте клиента и количество используемых им продуктов банка (-0.304). Необходимо переходить к подготовке данных.

## Подготовка данных

Изменим названия столбцов на нижний регистр, а также заменим пробелы в названиях переменных на нижнее подчеркивание:

In [6]:
df.columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography', 'gender',
             'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card', 'is_active_member',
             'estimated_salary', 'exited']

В таблице имеется признак `row_number`, который отвечает за индексацию наблюдений, однако при сохранении данных в переменной `df` индексация уже проставлена автоматически, поэтому можем избавиться от данного признака:

In [7]:
df = df.drop(['row_number'], axis=1)

Переменные типа `object` потребуют перекодирования, однако среди них есть переменная `surname`, перекодирование которой однозначно приведёт к появлению огромного количества ненужных столбцов, поэтому также исключим её из таблицы. Данное действие ни на что не повлияет. Также исключим переменную `customer_id`, которая не понадобится для модели машинного обучения, но может случайно искажать результаты.

In [8]:
df = df.drop(['surname', 'customer_id'], axis=1)

Также в данных имеют пропуски, а именно - в переменной `Tenure`, отвечающая за то, сколько лет человек является клиентом банка. Изучим подробнее тех клиентов, у кого в данном признаке стоит `NA`:

In [9]:
df[df['tenure'].isna()].head(8)

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
30,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
82,543,France,Female,36,NaN,0.00,2,0,0,26019.59,0
85,652,Spain,Female,75,NaN,0.00,2,1,1,114675.75,0
94,730,Spain,Male,42,NaN,0.00,2,0,1,85982.47,0


Как видно, значимых сходств у этих клиентов нет. Заполнять эти пропуски средним или медианой исказило бы результаты исследования, к тому же, у нас нет достоверной информации, почему эти пропуски появились. Предположим, что эти клиенты присоединились к банку недавно, поэтому они ещё не имеют такой информации. Следовательно, стоит заполнить пропуски нулями:

In [10]:
df['tenure'] = df['tenure'].fillna(0)

Теперь таблица находится в пригодном для построения модели виде, однако осталась нерешённой проблема с переменными типа `object`. Применим технику прямого кодирования One-Hot Encoding для двух переменных: `geography` и `gender`:

In [11]:
#при помощи метода pandas перекодируем категориальные переменные и исключаем первый столбец во избежании дамми-ловушки
df = pd.get_dummies(df, drop_first=True)

Посмотрим, как теперь выглядит наша таблица. Для этого выведем на экран первые 3 строки:

In [12]:
df.head(3)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0


Как видно, метод библиотеки разбил переменные `geography` и `gender` на несколько других признаков, а у нас не осталось в таблице категориальных переменных - всё, как и было необходимо.

### Вывод

На этапе подготовке данных мы привели столбцы в нужный регистр, избавились от ненужных для модели машинного обучения признаков. Также заполнили пропуски в признаке `tenure` нулями во избежании искажения результатов исследования. Категориальные переменные были перекодирования методом One-Hot Encoding, в результате чего в таблице появились дополнительные признаки, но исчезли категориальные переменные. Далее можем переходить к исследованию задачи исследования.

## Исследование задачи

В наших данных возможен дисбаланс классов: тех, кто покинул банк, существенно меньше, чем тех, кто банк не покидал - это обычная ситуация, свойственная для большего периода времени существования банка. Однако, для начала попробуем построить модель без учёта этого дисбаланса, а исследуем его позже. 

Разобьём наши данные на 3 выборки: обучающую, валидационную и тестовую в пропорции 3:1:1. Перед этим разделим наши признаки на целевой и те, которые будут влиять на целевой признак: 

In [13]:
features = df.drop('exited', axis=1)
target = df['exited']

#разбиваем features и target на три выборки в пропорции 60%:20%:20% и заданным случайное значение RANDOM_STATE
features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.2, random_state=RANDOM_STATE)
features_train, features_test, target_train, target_test = train_test_split(features_train, target_train, 
                                                                              test_size=0.25, random_state=RANDOM_STATE)

Удостоверимся, корректными ли получились размеры наших выборок:

In [14]:
print('Размер обучающей выборки равен:', target_train.shape)
print('Размер валидационной выборки равен:', target_valid.shape)
print('Размер тестовой выборки равен:', target_test.shape)

Размер обучающей выборки равен: (6000,)
Размер валидационной выборки равен: (2000,)
Размер тестовой выборки равен: (2000,)


Теперь необходимо исследовать различные модели и понять, какая из них окажется наиболее оптимальной для исследования. Нам предстоит выбрать подходящую модель классификации из трёх вариантов: дерево решений, лес решений, логистическая регрессия. Начнём построение моделей с дерева решений. Для этого воспользуемся специальным модулем библиотеки `sklearn`. Чтобы понять, какая глубина будет для дерева оптимальной, применим цикл:

In [15]:
#зададим нулевые гиперпараметры для лучшей модели, максимальной глубины дерева и лучшего accuracy_score
best_model_tree = None
best_depth_tree = 0
best_result_tree = 0
#с помощью цикла переберём модели по глубине дерева и выявим наилучшую по f1-мере
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = f1_score(target_valid, predictions)
    #наилучшую модель сохраняем
    if result > best_result_tree:
        best_model_tree = model
        best_depth_tree = depth
        best_result_tree = result
print('Наилучшее дерево решений имеет глубину:', best_depth_tree, "и её f1-мера составляет:", best_result_tree)

Наилучшее дерево решений имеет глубину: 9 и её f1-мера составляет: 0.5626666666666668


Наиболее оптимальное дерево получилось с глубиной 9 и f1-мерой в 0.562. В целом, это результат можно считать средним, однако выводы о качестве модели достаточно рано, ведь нужно проверить также другие варианты. Обратимся к лесу решений:

In [16]:
#зададим нулевые гиперпараметры для лучшей модели, максимальной глубины каждого дерева, критерия, количества деревьев и f1-меры
best_model_forest = None
best_depth_forest = 0
best_result_forest = 0
best_est_forest = 0
best_criterion_forest = None
#с помощью вложенного цикла переберём различные гиперпараметры и выявим наилучшую модель по f1-мере
for crit in ['gini', 'entropy']:
    for est in range(1, 30):
        for depth in range(1, 12):
            model = RandomForestClassifier(random_state=RANDOM_STATE, criterion=crit,
                                          max_depth=depth, n_estimators=est)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            result = f1_score(target_valid, predictions)
            #наилучшую модель сохраняем
            if result > best_result_forest:
                best_model_forest = model
                best_depth_forest = depth
                best_result_forest = result
                best_est_forest = est
                best_criterion_forest = crit
print('Наилучший лес решений имеет глубину:', best_depth_forest,',', 'критерий:', best_criterion_forest,',', 'число деревьев:',
     best_est_forest, 'и его f1-мера составляет:', best_result_forest)

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-sco

Наилучший лес решений имеет глубину: 11 , критерий: entropy , число деревьев: 17 и его f1-мера составляет: 0.5897810218978102


Получили, что для леса решений максимальная глубина каждого дерева составляет 11, деревьев 17 и критерий используется энтропический. При этом, f1-мера у леса оказывается выше, чем у дерева решений: 0.589, однако разница не слишком существена. Остаётся проверить только модель логистической регрессии:

In [17]:
model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs')
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
result = f1_score(target_valid, predictions)
print('Модель логистической регрессии имеет f1-меру:', result)

Модель логистической регрессии имеет f1-меру: 0.1273100616016427


Логистическая регрессия показала самые слабые результаты, f1-мера составила 0.127. 

### Вывод

На этапе построения модели без учёта дисбаланса классов построили 3 различные модели и по каждой измерили f1-меру:

* Дерево решений с глубиной 9: f1-мера 0.562
* Лес решений с глубиной каждого дерева 11, количеством деревьев 17 и критерием энтропии: f1-мера 0.589
* Логистическая регрессия: f1-мера 0.127

Следовательно, на данном этапе наилучшей оказалась модель леса решений, однако дерево решений получило практически те же результаты. Логистическая регрессия оказалась самой слабой моделью. Продолжим работу над моделями и постараемся улучшить их качество при помощи борьбы с дисбалансом классов и иных возможностей.

## Борьба с дисбалансом

Улучшим качество моделей, учитывая дисбаланс классов. Сначала необходимо посмотреть, что из себя представляет этот дисбаланс:

In [18]:
print('Количество клиентов, не покинувших банк:', df[df['exited'] == 0]['exited'].count())
print('Количество клиентов, покинувших банк:', df[df['exited'] == 1]['exited'].count())

Количество клиентов, не покинувших банк: 7963
Количество клиентов, покинувших банк: 2037


Видно, что клиентов, которые покинули банк, сильно меньше, чем тех, кто остался его клиентов - практически в 4 раза. Данная пропорция заставляет нас делать вывод о наличии дисбаланса классов. Воспользуемся техникой upsampling, которая искусственно увеличит количество тех, кто покинул банк - это улучшит качество построенных моделей. Напишем функцию, которая позволит это осуществить:

In [19]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=RANDOM_STATE)
    
    return features_upsampled, target_upsampled

Теперь вызовем эту функцию и увеличим количество клиентов, покинувших банк, в 3 раза, чтобы приблизиться к количество тех, кто банк не оставил, но при этом не делать это число существенно выше:

In [20]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 3)

Теперь можем обратиться к построенным моделям вновь и увидеть, как изменилось их качество с учётом поверженного дисбаланса классов. Начнём с дерева решений с заранее подобранными под него параметрами:

In [21]:
model = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=7)
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_valid)
result = f1_score(target_valid, predictions)
print('Наилучшее дерево решений имеет f1-меру:', best_result_tree)

Наилучшее дерево решений имеет f1-меру: 0.5626666666666668


Как видно, f1-мера дерева решений совсем не изменилась после того, как мы ввели баланс классов. Результат остался на том же уровне - 0.562. Проверим, отреагирует ли на изменение дерево решений:

In [22]:
model = RandomForestClassifier(random_state=RANDOM_STATE, criterion='gini',
                                max_depth=10, n_estimators=23)
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_valid)
result = f1_score(target_valid, predictions)
print('Наилучший лес решений имеет глубину:', best_depth_forest,',', 'критерий:', best_criterion_forest,',', 'число деревьев:',
     best_est_forest, 'и его f1-мера составляет:', best_result_forest)

Наилучший лес решений имеет глубину: 11 , критерий: entropy , число деревьев: 17 и его f1-мера составляет: 0.5897810218978102


Дерево решений также осталось без изменений. Остаётся проверить только логистическую регрессию:

In [23]:
model = LogisticRegression(random_state=RANDOM_STATE, solver='lbfgs')
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_valid)
result = f1_score(target_valid, predictions)
print('Модель логистической регрессии имеет f1-меру:', result)

Модель логистической регрессии имеет f1-меру: 0.39366963402571714


Видим, что модель логистической регрессии отреагировала на изменения, о чём говорит увеличение её f1-меры: 0.393 вместо 0.127, однако результаты всё ещё далеки от идеальных. Попробуем поработать с наилучшей на текущий момент моделью - дерево решений и подберём ему новые оптимальные критерии

In [24]:
#зададим нулевые гиперпараметры для лучшей модели, максимальной глубины каждого дерева, критерия, количества деревьев и f1-меры
best_model_forest = None
best_depth_forest = 0
best_result_forest = 0
best_est_forest = 0
best_criterion_forest = None
#с помощью вложенного цикла переберём различные гиперпараметры и выявим наилучшую модель по f1-мере
for crit in ['gini', 'entropy']:
    for est in range(1, 30):
        for depth in range(1, 12):
            model = RandomForestClassifier(random_state=RANDOM_STATE, criterion=crit,
                                          max_depth=depth, n_estimators=est)
            model.fit(features_upsampled, target_upsampled)
            predictions = model.predict(features_valid)
            result = f1_score(target_valid, predictions)
            #наилучшую модель сохраняем
            if result > best_result_forest:
                best_model_forest = model
                best_depth_forest = depth
                best_result_forest = result
                best_est_forest = est
                best_criterion_forest = crit
print('Наилучший лес решений имеет глубину:', best_depth_forest,',', 'критерий:', best_criterion_forest,',', 'число деревьев:',
     best_est_forest, 'и его f1-мера составляет:', best_result_forest)

Наилучший лес решений имеет глубину: 10 , критерий: gini , число деревьев: 26 и его f1-мера составляет: 0.6203703703703705


Получили новые результаты, а также увеличили f1-меру до 0.620. С этим результатом можем идти далее и проводить тестирование модели на тестовой выборке, однако мы попробуем также другой метод борьбы с дисбалансом классов, чтобы проверить, был ли выбран самый оптимальный. Воспользуемся противоположной техникой - downsampling. Напишем функцию, при помощи которой сократим количество оставшихся в банке в 3 раза:

In [25]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=RANDOM_STATE)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=RANDOM_STATE)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=RANDOM_STATE)
    
    return features_downsampled, target_downsampled

Теперь воспользуемся данной функцией:

In [26]:
features_upsampled, target_upsampled = downsample(features_train, target_train, 0.33)

Вновь обучим лес решений и проверим, какие результаты получаются на этот раз:

In [27]:
#зададим нулевые гиперпараметры для лучшей модели, максимальной глубины каждого дерева, критерия, количества деревьев и f1-меры
best_model_forest = None
best_depth_forest = 0
best_result_forest = 0
best_est_forest = 0
best_criterion_forest = None
#с помощью вложенного цикла переберём различные гиперпараметры и выявим наилучшую модель по f1-мере
for crit in ['gini', 'entropy']:
    for est in range(1, 30):
        for depth in range(1, 12):
            model = RandomForestClassifier(random_state=RANDOM_STATE, criterion=crit,
                                          max_depth=depth, n_estimators=est)
            model.fit(features_upsampled, target_upsampled)
            predictions = model.predict(features_valid)
            result = f1_score(target_valid, predictions)
            #наилучшую модель сохраняем
            if result > best_result_forest:
                best_model_forest = model
                best_depth_forest = depth
                best_result_forest = result
                best_est_forest = est
                best_criterion_forest = crit
print('Наилучший лес решений имеет глубину:', best_depth_forest,',', 'критерий:', best_criterion_forest,',', 'число деревьев:',
     best_est_forest, 'и его f1-мера составляет:', best_result_forest)

Наилучший лес решений имеет глубину: 6 , критерий: entropy , число деревьев: 22 и его f1-мера составляет: 0.5835140997830803


Наивысшая f1-мера при лучших гиперпараметрах оказалась равна 0.583. Значит, техника upsampling показала себя лучше противоположной, поэтому и модель будем применять именно после первой техники борьбы с дисбалансом классов.

### Вывод

После борьбы с дисбалансом классов удалось увеличить f1-меру лишь у модели логистической регрессии: она составила 0.393 вместо прежней 0.127, однако модели дерева решений и случайного леса сохранили показатели. В ходе изменения гиперпараметров модели случайного леса удалось достичь лучших результатов при гиперпараметрах глубины 10, критерия Джини, 26 деревьях. F1-мера такого леса составляет 0.620. Можно переходить к тестированию модели.

## Тестирование модели

Протестируем модель случайного леса на тестовой выборке, чтобы убедиться, что модель оказалась удачной:

In [28]:
model = RandomForestClassifier(random_state=RANDOM_STATE, criterion='gini', max_depth=10, n_estimators=26)
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_test)
result = f1_score(target_test, predictions)
print('F1-мера случайного леса на тестовой выборке составила:', result)

F1-мера случайного леса на тестовой выборке составила: 0.6179039301310043


Как видно, f1-мера оказалась немного ниже, чем была прежде: 0.617 вместо 0.620. Тем не менее, это говорит о том, что модель имеет достаточно высокое качество. Заказчик ожидал увидеть данный показатель выше 0.59, что и было выполнено. В завершении посмотрим на меру AUC-ROC выбранной модели. 

In [29]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

auc_roc = roc_auc_score(target_test, probabilities_one_test)
print('F1-мера модели составила:', result, 'при значении AUC-ROC:', auc_roc)

F1-мера модели составила: 0.6179039301310043 при значении AUC-ROC: 0.8570917473156019


### Вывод

Тестирование модели прошло успешно, и модель случайного леса показала наилучшие результаты для заказчика. При ожидаемой f1-мере более 0.59 мы получили 0.617 на тестовой выборке, что говорит о достаточно качественной моделе. Также была изучена метрика AUC-ROC, которая составила 0.857, что вновь говорит о хорошо составленной моделе. На этом исследование завершается, и можно переходить к окончательным выводам.

## Вывод

Цель исследования заключалась в построении модели, предсказывающей уход клиентов из банка. После предварительного изучения данных было выдвинуто предположение о наличии дисбаланса классов, однако изначально модели были построены без работы над этой проблемой. Основной метрикой модели данного исследования была выбрана f1-мера, и результаты при первичных моделях были следующие:

* Дерево решений с глубиной 9: f1-мера 0.562
* Лес решений с глубиной каждого дерева 11, количеством деревьев 17 и критерием энтропии: f1-мера 0.589
* Логистическая регрессия: f1-мера 0.127

Однако после борьбы с дисбалансом классов удалось достичь иных результатов. В частности, модель логистической регресии повысила свою метрику f1, однако этого оказалось недостаточно для дальнейшей работы над моделью, поэтому была выбрана модель случайного леса и после изменения гиперпараметров удалось достичь наилучших результатов: f1-мера модели на тестовой выборке составила 0.617, а метрика AUC-ROC - 0.852. Таким образом, данная модель является наиболее пригодной для заказчика и поможет ему вовремя определить, кто из клиентов в скором времени может отказаться от услуг банка.